# Introduction
This notebook will contain code for quickly and procedurally running experiments and comparing their results.

In [3]:
# package imports
import sys
import os
import subprocess
import numpy as np
from itertools import product
from skopt import gp_minimize

# project imports
sys.path.insert(0, os.path.abspath('..'))
from optimization_utils.xml_generate_utils import render_xml
from optimization_utils.result_utils import get_madymo_summary, get_dfs_rmse

In [4]:
# arguments for running in local environment
# notice that the variable name is capitalized, by convention,
# because we want to indicate that we shouldn't ever change its value
PATH_TO_DATA = fr'../data'
REAL_WORLD_DF = get_madymo_summary(fr"{PATH_TO_DATA}/IIHS_lac.csv")

In [5]:
# enumerate some experimentional arguments to try
rangeHR = map(str, np.arange(0.0, 1.2, 0.2).tolist())
rangeSeat = map(str, np.arange(0.0, 1.2, 0.2).tolist())
experiment_dicts = [dict(zip(('Friction_Headrest','Friction_Seat'), (i,j))) for i,j in product(rangeHR,rangeSeat)]

In [6]:
# create a minimization function wrapper
def sim_rmse_function(dimension_names,
        reference_csv = REAL_WORLD_DF,
        defines_xml_tempate_path = fr"{PATH_TO_DATA}/Defines.xml",
        defines_xml_output_path = fr"{PATH_TO_DATA}/experiment.xml",
        simulation_xml_path= fr"{PATH_TO_DATA}/Madymo.xml"):
    """
    Return a function to return the RMSE between a simulation parameter input dict and the real world data
    """
    def get_rmse(args):
        """
        create an rmse function to be returned for optimization
        """
        os.chdir(PATH_TO_DATA) # TODO changing directory into path for madymo.xml, refactor code to not need this
        
        # generate the XML which will be used to run the experiment
        experiment_dict = zip(dimension_names, args)
        render_xml(defines_xml_tempate_path, defines_xml_output_path, experiment_dict)
        
        # call the madymo subprocess to generate experimental results
        subprocess.call(['madymo20201', '-3d', fr"{PATH_TO_DATA}/Madymo.xml"])
        
        # read the experimental data spit out by Madymo and compare to the real data
        experimental_df = get_madymo_summary("Madymo_lac.csv")
        return get_dfs_rmse(reference_csv, experimental_df)
        
        
    # return the generated function
    return get_rmse

In [ ]:
# Run the optimizer to sample function space for good inputs, and print a summary
dimensions = [
    {'name': 'Friction_Headrest', 'min': 0, 'max': 1.2},
    {'name': 'Friction_Seat', 'min': 0, 'max': 1.2}
]

results = gp_minimize(
        func = sim_rmse_function([d['name'] for d in dimensions]),
        dimensions = [(d['min'], d['max']) for d in dimensions],
        n_initial_points = 10,
        n_calls=25,
        x0 = [.5, .5, .5],
        random_state = 100)

print(f"""
results summary:
best inputs found:\t{results.x}
trial count:\t{len(results.func_vals)}
""")

best_input = results.x
best_input_dict = zip([d['name'] for d in dimensions], best_input)

In [ ]:
#Re-run the optimized run
render_xml(
        fr"{PATH_TO_DATA}/Defines.xml",
        fr"{PATH_TO_DATA}/experiment.xml",
        min_dict)
subprocess.call(['madymo20201', '-3d', fr"{PATH_TO_DATA}/Madymo.xml"])
experimental_df = get_madymo_summary("Madymo_lac.csv")

In [ ]:
#IIHS_list = experimental_df.values
#Mad_list = REAL_WORLD_DF.values
#IIHS_HA = [IIHS_list[:0],IIHS_list[:1]]
#Mad_HA = [Mad_list[0], Mad_list[0]]

#print(IIHS_list[2:])

ex_list = list(experimental_df.columns)
Mad_list = list(REAL_WORLD_DF.columns)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(experimental_df[ex_list[0]], experimental_df[ex_list[1]], 'r--', label='Experimental Head Accel')
ax.plot(REAL_WORLD_DF[Mad_list[0]], REAL_WORLD_DF[Mad_list[1]], 'r', label='IIHS Head Accel')
ax.plot(experimental_df[ex_list[0]], experimental_df[ex_list[5]], 'b--', label='Experimental Pelvis Accel')
ax.plot(REAL_WORLD_DF[Mad_list[0]], REAL_WORLD_DF[Mad_list[5]], 'b', label='IIHS Pelvis Accel')
plt.xlabel('Time - sec')
plt.ylabel('Acceleration - m/s/s')
leg = ax.legend();